In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#!pip install pytorch-lightning

In [11]:
import pytorch_lightning as pl
import pandas as pd
import cv2
import os 
from torch import nn
from torch.utils.data import Dataset ,DataLoader, random_split
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import torchmetrics
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger

In [4]:
pl.seed_everything(7)
PATH_DATASETS = os.environ.get("PATH_DATASETS", "/content/drive/MyDrive/Datasets")
BATCH_SIZE = 256 if torch.cuda.is_available() else 64

Global seed set to 7


In [19]:
class KMNISTDatamodule(pl.LightningDataModule):
  def __init__(self, data_dir: str= PATH_DATASETS, batch_size: int = BATCH_SIZE):
    super().__init__()
    self.data_dir = data_dir
    self.batch_size = batch_size
    self.transform = {
        'train':transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ]),

        'test':transforms.Compose([
            transforms.ToTensor()
        ]),
    }
    self.dims = (1, 28, 28)
    self.num_classes = 10

  def prepare_data(self):
    datasets.KMNIST(self.data_dir, train=True, download=True)
    datasets.KMNIST(self.data_dir, train=False, download=True, transform=self.transform['test'])

  def setup(self, stage=None):
    if stage=='fit' or stage is None:
      kmnist_full = datasets.KMNIST(self.data_dir, train=True, transform=self.transform['train'])
      self.kmnist_train, self.kmnist_val = random_split(kmnist_full, [55000,5000])
    
    if stage=='test' or stage is None:
      self.kmnist_test = datasets.KMNIST(self.data_dir, train=False, transform=self.transform['test'])

  def train_dataloader(self):
    return DataLoader(self.kmnist_test, batch_size=self.batch_size)

  def val_dataloader(self):
    return DataLoader(self.kmnist_val, batch_size=self.batch_size)

  def test_dataloader(self):
    return DataLoader(self.kmnist_test, batch_size=self.batch_size)

  def visualize(self):
    figure = plt.figure(figsize=(10,8))
    cols, rows = 5, 5 
    for i in range(1, cols * rows + 1):
      sample_idx = torch.randint(len(self.kmnist_train), size=(1,)).item()
      img, label = self.kmnist_train[sample_idx]
      plt.subplot(rows, cols, i)
      plt.title(label)
      plt.axis('off')
      plt.imshow(img.squeeze(), cmap='gray')
    plt.show()

In [24]:
class LitKMNISTClassifier(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
    self.layer1 = nn.Sequential(self.conv1, nn.ReLU(), nn.MaxPool2d(kernel_size=2))
    self.layer2 = nn.Sequential(self.conv2, nn.ReLU(), nn.MaxPool2d(kernel_size=2))
    self.out = nn.Linear(32*7*7, 10)
    self.save_hyperparameters()

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)

    x = x.view(x.size(0), -1)
    output = self.out(x)

    return output

  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self.forward(x)
    loss = nn.functional.cross_entropy(logits, y)
    self.log('train loss', loss)
    return loss

  def evaluate(self, batch, stage=None):
    x, y = batch
    logits = self.forward(x)
    loss = nn.functional.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    acc = accuracy(preds, y)

    if stage:
      self.log(f"{stage}_loss", loss, prog_bar=True)
      self.log(f"{stage}_acc", acc, prog_bar=True)

  def validation_step(self, batch, batch_idx):
    self.evaluate(batch, "val")

  def test_step(self, batch, batch_idx):
    self.evaluate(batch, "test")

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-2)

In [25]:
dm = KMNISTDatamodule()
dm.prepare_data()

In [26]:
dm.setup()

In [27]:
model = LitKMNISTClassifier()
trainer = pl.Trainer(max_epochs=30,
                  default_root_dir="/content/drive/MyDrive/Checkpoints", 
                  accelerator="auto",
                  devices=1 if torch.cuda.is_available() else None,
                  callbacks=[LearningRateMonitor(logging_interval="step"),
                                TQDMProgressBar(refresh_rate=10)],
)
trainer.fit(model, dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | conv1  | Conv2d     | 416   
1 | conv2  | Conv2d     | 12.8 K
2 | layer1 | Sequential | 416   
3 | layer2 | Sequential | 12.8 K
4 | out    | Linear     | 15.7 K
--------------------------------------
28.9 K    Trainable params
0         Non-trainable params
28.9 K    Total params
0.116     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1895: PossibleUserWarning: The number of training batches (40) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.


In [28]:
trainer.test(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9801999926567078
        test_loss          0.057156555354595184
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.9801999926567078, 'test_loss': 0.057156555354595184}]